# LSTM

In [13]:
from sklearn.metrics import confusion_matrix
import pickle
import keras
from keras.layers import LSTM
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes):
    x_train = x_train.reshape(-1, n_step, n_input)
    x_test = x_test.reshape(-1, n_step, n_input)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, n_classes)
    y_test = keras.utils.to_categorical(y_test, n_classes)
    return (x_train, x_test, y_train, y_test)

def lstm_model(n_input, n_step, n_hidden, n_classes):
    model = Sequential()
    model.add(LSTM(n_hidden, batch_input_shape=(None, n_step, n_input), unroll=True))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    return model

def train_lstm(model, x_train, y_train, x_test, y_test, 
        learning_rate, training_iters, batch_size):
    adam = Adam(lr=learning_rate)
    model.summary()
    model.compile(optimizer=adam,
        loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train,
        batch_size=batch_size, epochs=training_iters,
        verbose=1, validation_data=(x_test, y_test))

def print_result(data, x_train, x_test, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(x_train)
    test_pred = model.predict_classes(x_test)
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

def mnist_lstm_main():
    # training parameters
    learning_rate = 0.001
    training_iters = 20
    batch_size = 20

    # model parameters
    n_input = 40
    n_step = 10
    n_hidden = 256
    n_classes = 10

    data = load_pkl('label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl')
    x_train, y_train, x_test, y_test = data['train_gaf'], data['train_label'][:, 0], data['test_gaf'], data['test_label'][:, 0]
    x_train, x_test, y_train, y_test = lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes)

    model = lstm_model(n_input, n_step, n_hidden, n_classes)
    train_lstm(model, x_train, y_train, x_test, y_test, learning_rate, 
               training_iters, batch_size)
    scores = model.evaluate(x_test, y_test, verbose=0)
    print('LSTM test accuracy:', scores[1])
    print_result(data, x_train, x_test, model)

如果把Epoch改成20，Batch Size改成20，可以把Accuracy提升到0.839

In [14]:
mnist_lstm_main()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               304128    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 306,698
Trainable params: 306,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
750/750 [==============================] - 19s 24ms/step - loss: 1.8860 - accuracy: 0.2594 - val_loss: 1.3246 - val_accuracy: 0.4098
Epoch 2/20
750/750 [==============================] - 17s 23ms/step - loss: 1.2386 - accuracy: 0.4855 - val_loss: 0.7120 - val_accuracy: 0.7310
Epoch 3/20
750/750 [==============================] - 17s 23ms/step - loss: 0.7856 - accur

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[1762  167  129  199  105  137  241  146  114]
 [  36 1458    0    4    0    0    2    0    0]
 [ 101    0 1368    0   31    0    0    0    0]
 [   9   56    0 1410    0    0    0   25    0]
 [  75    0    6    0 1328    0    2    0   89]
 [  36    4    0   13    0 1256    3  188    0]
 [  17    2    0    0    1    0 1436    0   44]
 [   2    5    0  493    0    0    0 1000    0]
 [  30    0    1    0  207    0   29    0 1233]] 

 [[592  53  50  61  40  43  76  50  35]
 [  3 497   0   0   0   0   0   0   0]
 [ 17   0 477   0   6   0   0   0   0]
 [  6  14   0 476   0   0   0   4   0]
 [ 29   0   4   0 440   0   0   0  27]
 [ 10   0   0   2   0 446   0  42   0]
 [  6   0   0   0   0   0 484   0  10]
 [  1   1   0 151   0   0   0 347   0]
 [  7   0   0   0  48   0   8   0 437]]


# CNN

In [16]:
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle

from keras import backend as K
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Activation, MaxPool2D


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def get_cnn_model(params):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(10, 10, 4)))
    model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    return model

def train_model(params, data):
    model = get_cnn_model(params)
    model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])
    hist = model.fit(x=data['train_gaf'], y=data['train_label_arr'],
                     batch_size=params['batch_size'], epochs=params['epochs'], verbose=2)
    return (model, hist)

def print_result(data, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(data['train_gaf'])
    test_pred = model.predict_classes(data['test_gaf'])
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

調了Learning Rate, Epochs 以及Batch_size, 最後得到最好的結果是0.845799

In [20]:
PARAMS = {}
PARAMS['pkl_name'] = 'label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl'
PARAMS['classes'] = 9
PARAMS['lr'] = 0.001
PARAMS['epochs'] = 40
PARAMS['batch_size'] = 40
PARAMS['optimizer'] = optimizers.SGD(lr=PARAMS['lr'])

# ---------------------------------------------------------
# load data & keras model
data = load_pkl(PARAMS['pkl_name'])
# train cnn model
model, hist = train_model(PARAMS, data)
# train & test result
scores = model.evaluate(data['test_gaf'], data['test_label_arr'], verbose=0)
print('CNN test accuracy:', scores[1])
print_result(data, model)

Epoch 1/40
375/375 - 7s - loss: 2.1400 - accuracy: 0.1806
Epoch 2/40
375/375 - 7s - loss: 1.9204 - accuracy: 0.2773
Epoch 3/40
375/375 - 7s - loss: 1.6248 - accuracy: 0.4121
Epoch 4/40
375/375 - 7s - loss: 1.3993 - accuracy: 0.5145
Epoch 5/40
375/375 - 7s - loss: 1.1936 - accuracy: 0.5974
Epoch 6/40
375/375 - 7s - loss: 1.0113 - accuracy: 0.6689
Epoch 7/40
375/375 - 7s - loss: 0.8838 - accuracy: 0.6985
Epoch 8/40
375/375 - 7s - loss: 0.7985 - accuracy: 0.7289
Epoch 9/40
375/375 - 7s - loss: 0.7385 - accuracy: 0.7445
Epoch 10/40
375/375 - 7s - loss: 0.6940 - accuracy: 0.7584
Epoch 11/40
375/375 - 7s - loss: 0.6563 - accuracy: 0.7725
Epoch 12/40
375/375 - 7s - loss: 0.6257 - accuracy: 0.7787
Epoch 13/40
375/375 - 7s - loss: 0.5995 - accuracy: 0.7903
Epoch 14/40
375/375 - 7s - loss: 0.5776 - accuracy: 0.7976
Epoch 15/40
375/375 - 7s - loss: 0.5561 - accuracy: 0.8033
Epoch 16/40
375/375 - 7s - loss: 0.5408 - accuracy: 0.8103
Epoch 17/40
375/375 - 7s - loss: 0.5263 - accuracy: 0.8158
Epoch 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[2359   67  105   98   53   60  114   68   76]
 [  82 1408    0    9    0    0    1    0    0]
 [  85    0 1407    0    8    0    0    0    0]
 [  74   32    0 1323    0    0    0   71    0]
 [ 105    0   28    0 1012    0    7    0  348]
 [ 172    0    0    2    0 1296    2   28    0]
 [  82    2    2    0    0    0 1378    0   36]
 [  73    3    0  184    0    8    0 1232    0]
 [  67    0    2    0   60    0   55    0 1316]] 

 [[782  22  43  32  20  20  26  25  30]
 [ 18 478   0   4   0   0   0   0   0]
 [ 26   0 474   0   0   0   0   0   0]
 [ 37  10   0 438   0   0   0  15   0]
 [ 59   0   9   0 319   0   1   0 112]
 [ 74   0   0   0   0 420   0   6   0]
 [ 40   0   1   0   0   0 454   0   5]
 [ 29   1   0  43   0   3   0 424   0]
 [ 37   0   0   0   4   0  19   0 440]]


# Reference 

[Peculab Github](https://github.com/pecu/FinTech_CommonWealth_Magazine/tree/master/Financial_Innovation/FiancailVision/HW2_ID_%E5%A7%93%E5%90%8D)